In [2]:
"""
Script de Inferencia para el Modelo de Calidad del Aire.

Este script carga el modelo XGBoost entrenado y lo utiliza para predecir la
calidad del aire a partir de nuevos datos de entrada proporcionados por el usuario.
"""

# 1. IMPORTACIÓN DE LIBRERÍAS
import pandas as pd
import numpy as np
import joblib
from pathlib import Path

In [3]:
# 2. CONFIGURACIÓN Y CONSTANTES
# Definir rutas de entrada. Usar Path es más seguro y portable entre sistemas operativos.
BASE_PATH = Path(r"B:\Documentos\Universidad\Universidad 2025\Primavera\Machine Learning\segundo intento modelo ML\Codigos")
MODEL_PATH = BASE_PATH / "modelo_xgboost_final.pkl"

# Diccionarios para interpretar la salida del modelo
INTERVALOS_PM25 = {
    0: "0 - 12 µg/m³ (Muy Bueno)",
    1: "13 - 35 µg/m³ (Bueno)",
    2: "36 - 55 µg/m³ (Regular)",
    3: "56 - 150 µg/m³ (Malo)",
    4: "> 150 µg/m³ (Muy Malo)"
}

NOMBRES_CLASES = ['Muy Bueno', 'Bueno', 'Regular', 'Malo', 'Muy Malo']

# Se define explícitamente el orden de las columnas que espera el modelo.
# Esto evita errores si el diccionario de entrada se crea en un orden diferente.
FEATURE_ORDER = [
    'CO', 'Humedad', 'NO', 'NO2', 'NOx', 'O3', 'PM10', 'SO2', 'Temp',
    'Viento_Cos', 'Viento_Sin'
]

In [4]:
# 3. FUNCIONES AUXILIARES
def cargar_modelo(path_modelo):
    """
    Carga el modelo serializado desde la ruta especificada.
    Verifica si el archivo existe antes de intentar cargarlo.
    """
    if not path_modelo.exists():
        raise FileNotFoundError(
            f"¡Error! No se encuentra el archivo del modelo en: {path_modelo}\n"
            "Asegúrate de haber ejecutado el script de entrenamiento primero."
        )
    print(f"Cargando modelo desde: {path_modelo}...")
    return joblib.load(path_modelo)

def predecir_calidad_aire(datos_entrada, modelo_entrenado):
    """
    Realiza una predicción de calidad del aire a partir de un diccionario de datos.

    Args:
        datos_entrada (dict): Diccionario con los valores de los sensores.
        modelo_entrenado: El objeto del modelo de clasificación cargado.

    Returns:
        tuple: Contiene la etiqueta de la clase, la confianza y el intervalo estimado.
    """
    # A. Preparar los datos de entrada (Ingeniería de Características)
    # Replicar la transformación de la dirección del viento a componentes vectoriales.
    wd_rad = np.deg2rad(datos_entrada['Viento_Grados'])
    v_cos = np.cos(wd_rad)
    v_sin = np.sin(wd_rad)

    # Crear un diccionario con todos los datos para el DataFrame.
    input_data = {
        'CO': datos_entrada['CO'], 'Humedad': datos_entrada['Humedad'],
        'NO': datos_entrada['NO'], 'NO2': datos_entrada['NO2'],
        'NOx': datos_entrada['NOx'], 'O3': datos_entrada['O3'],
        'PM10': datos_entrada['PM10'], 'SO2': datos_entrada['SO2'],
        'Temp': datos_entrada['Temp'], 'Viento_Cos': v_cos, 'Viento_Sin': v_sin
    }
    # Crear el DataFrame asegurando el orden correcto de las columnas.
    input_df = pd.DataFrame([input_data], columns=FEATURE_ORDER)

    # B. Realizar la predicción
    pred_clase = modelo_entrenado.predict(input_df)[0]
    pred_proba = modelo_entrenado.predict_proba(input_df)[0]

    # C. Interpretar los resultados
    confianza = pred_proba[pred_clase] * 100
    etiqueta = NOMBRES_CLASES[pred_clase]
    intervalo = INTERVALOS_PM25[pred_clase]

    return etiqueta, confianza, intervalo

In [5]:
# 4. EJECUCIÓN PRINCIPAL
# El bloque 'if __name__ == "__main__"' es una buena práctica en Python.
# Asegura que el código solo se ejecute cuando el script es el archivo principal.
if __name__ == '__main__':
    try:
        # Cargar el modelo entrenado
        mi_modelo = cargar_modelo(MODEL_PATH)

        # =================================================================
        # == INGRESA AQUÍ LOS VALORES PARA LA PREDICCIÓN ==
        # =================================================================
        valores_usuario = {
            'CO': 2.5,
            'Humedad': 85.0,
            'NO': 20.0,
            'NO2': 40.0,
            'NOx': 60.0,
            'O3': 5.0,
            'PM10': 12.0, # <-- Valor clave para la predicción
            'SO2': 4.0,
            'Temp': 8.5,
            'Viento_Grados': 45
        }
        # =================================================================

        # Ejecutar la predicción
        resultado, seguridad, rango_estimado = predecir_calidad_aire(valores_usuario, mi_modelo)

        # =================================================================
        # == REPORTE FINAL ==
        # =================================================================
        print(f"\n{'=' * 50}")
        print("  RESULTADO DE LA PREDICCIÓN DE CALIDAD DEL AIRE")
        print(f"{'=' * 50}")
        print(f"Valores Ingresados: PM10={valores_usuario['PM10']}, Temp={valores_usuario['Temp']}°C")
        print(f"{'-' * 50}")
        print(f"  - Calidad del Aire Estimada: [{resultado.upper()}]")
        print(f"  - Confianza del Modelo:      {seguridad:.2f}%")
        print(f"  - Rango de PM2.5 Esperado:   {rango_estimado}")
        print(f"{'=' * 50}")

        if resultado in ['Malo', 'Muy Malo']:
            print("\n⚠️  ALERTA: La calidad del aire es perjudicial. Se recomienda evitar actividades al aire libre.")
        else:
            print("\n✅  La calidad del aire es segura.")

    except Exception as e:
        # Captura cualquier error (ej. archivo no encontrado) y lo muestra de forma clara.
        print(f"\n[ERROR] Ocurrió un problema durante la ejecución: {e}")

Cargando modelo desde: B:\Documentos\Universidad\Universidad 2025\Primavera\Machine Learning\segundo intento modelo ML\Codigos\modelo_xgboost_final.pkl...

  RESULTADO DE LA PREDICCIÓN DE CALIDAD DEL AIRE
Valores Ingresados: PM10=12.0, Temp=8.5°C
--------------------------------------------------
  - Calidad del Aire Estimada: [MUY BUENO]
  - Confianza del Modelo:      99.87%
  - Rango de PM2.5 Esperado:   0 - 12 µg/m³ (Muy Bueno)

✅  La calidad del aire es segura.
